## Future 简介
摘抄自《scala程序设计（第2版）》————2.5.3Future 简介

隐含参数可以代替参数默认值,而且更加灵活。我们这就来研究一个 Scala 库中使用隐含参数的例子 Future 。

scala.concurrent.Future 是 Scala 提供的一个并发工具,其中的 API 使用隐含参数来减少冗余代码。Akka 使用了Future ,但如果你并不需要 actor 的所有功能,也可以单独使用 Akka 中的 Future 部分。

当你将任务封装在 Future 中执行时,该任务的执行是异步的。 Future API 提供了多种机制去获取执行结果,如提供回调函数。当结果就绪时,回调函数将被调用。我们在这里就使用回调函数作为例子,关于其他 API 的讨论将推迟到第 17 章。

以下示例并发发出 5 个任务,并在任务结束时处理任务返回的结果:
```scala
// src/main/scala/progscala2/typelessdomore/futures.sc
import scala.concurrent.Future
import scala.concurrent.ExecutionContext.Implicits.global

def sleep(millis: Long) = {
  Thread.sleep(millis)
}

// Busy work ;)
// 繁忙的处理工作; )
def doWork(index: Int) = {
  sleep((math.random * 1000).toLong)
  index
}

(1 to 5) foreach { index =>
  val future = Future {
    doWork(index)
  }
  future onSuccess {
    case answer: Int => println(s"Success! returned: $answer")
  }
  future onFailure {
    case th: Throwable => println(s"FAILURE! returned: $th")
  }
}

sleep(1000)  // Wait long enough for the "work" to finish.
             // 等待足够长的时间,以确保工作线程结束。
println("Finito!")
```
在 import 语句之后的 sleep 函数中,我们调用 Thread 的 sleep 方法来模拟程序进行繁忙的处理工作,参数为睡眠的时长。 doWork 方法是对 sleep 的简单封装,传入一个 0 到 1000范围的随机数,表示睡眠的毫秒数。

接 下 来 的 部 分 就 有 趣 了。 我 们 用 foreach 对 一 个 从 1 到 5 的 Range 进 行 迭 代, 调 用 了scala.concurrent.Future.apply,这是单例对象 Future 的“工厂”方法。在这个例子中, Future.apply 传入了一个匿名函数,表示需要做的任务。我们用花括号而不是圆括号包围传入的匿名函数:
```scala
val future = Future {
  doWork(index)
}
```

Future.apply 返回一个新的 Future 对象,然后控制权就交还给循环了,该对象将在另一个线程中执行 doWork(index) 。接着,我们用 onSuccess 注册一个回调函数,当 future 成功执行完毕后,该回调会被执行。这个回调函数是一个偏函数。

类似地,我们用 onFailure 注册了一个回调函数来处理错误。回调函数将错误封装在一个 Throwable 的对象中。

最后,我们调用了 sleep ,以等待所有的任务执行完毕。
可能的运行结果如下所示。在本例中结果将以随机的顺序出现,这一点你也许有预期到:
```scala
$ scala src/main/scala/progscala2/typelessdomore/futures.sc
Success! returned: 1
Success! returned: 3
Success! returned: 4
Success! returned: 5
Success! returned: 2
Finito!
```

好了,上面说的这些与隐含参数有什么关系呢?从第二条 import 语句中可以发现答案:
```scala
import scala.concurrent.ExecutionContext.Implicits.global
```

我之前说过,每个 future 运行在各自独立的线程中,但这个说法不够严谨。事实上,Future API 允许我们通过 ExecutionContext 来配置并发操作的执行。上述 import 语句导入了默认的 ExecutionContext ,使用 Java 的 ForkJoinPool 设置来管理 Java 线程池。在示例中我们调用了 3 个方法,其中这些方法的第二个参数列表具有隐含的 ExecutionContext 参数。由于我们没有明显地指定第二个参数列表,系统使用了默认的 ExecutionContext 。

Apply 方法就是上述 3 个方法之一,以下是 apply 在 Future.apply 的 Scaladoc中的声明:
```scala
apply[T](body: => T)(implicit executor: ExecutionContext): Future[T]
```

注意第二个参数列表中有 implicit 关键字。以下是其他两个方法在 Fure.onSuccess 和 Future.onFailure 的 Scaladoc 中的声明:
```scala
def onSuccess[U](func: (Try[T]) => U)(
  implicit executor: ExecutionContext): Unit

def onFailure[U](callback: PartialFunction[Throwable, U])(
  implicit executor: ExecutionContext): Unit
```

Try 结构是一个处理 try {...} catch {...} 语句的工具,我们以后再讨论它。

那 么, 如 何 将 某 个 值 声 明 为 implicit 呢? 导 入 的 scala.concurrent.ExecutionContext.Implicits.global 是在 Future 中常用的默认 ExecutionContext 。它使用 implicit 关键字声明,因此如果调用时未显式给出 ExecutionContext 参数,编译器就会使用这个默认值,本例就是这种情况。只有由 implicit 关键字声明的,在当前作用域可见的对象才能用作隐含值;只有被声明为 implicit 的函数参数才允许调用时不给出实参,而采用隐含的值。

以下就是 Scala 源代码中的 scala.concurrent.ExecutionContext 对 Implicits.global 的声明。这段代码演示了如何用 implicit 关键字声明一个隐含的值(这里只给出了代码片段,省略了具体细节):
```scala
object Implicits {
  implicit val global: ExecutionContextExecutor =
    impl.ExecutionContextImpl.fromExecutor(null: Executor)
}
```